In [3]:
!nvidia-smi


Tue Apr  1 16:11:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [5]:
!pip install ultralytics

from IPython.display import Image

import ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.9/976.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [6]:
import random
import shutil
import tensorflow as tf

In [8]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Yp3RK5raZJ7EW8cRulik")
project = rf.workspace("hyzentech").project("breastcancerv2")
version = project.version(1)
dataset = version.download("folder")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to BreastCancerv2-1 in folder:: 100%|██████████| 5297/5297 [00:00<00:00, 5379.73it/s]


In [9]:
from pathlib import Path

main_path = Path("/content/BreastCancerv2-1")

image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

for split in ["train", "test", "valid"]:
    split_path = main_path / split
    if not split_path.exists():
        print(f"Skipping {split} as it does not exist.")
        continue

    print(f"\nSplit: {split}")
    total_images_in_split = 0

    for class_folder in split_path.iterdir():
        if class_folder.is_dir():
            num_images = sum(1 for file in class_folder.rglob("*") if file.suffix.lower() in image_extensions)
            total_images_in_split += num_images
            print(f"  Class: {class_folder.name} -> {num_images} images")

    print(f"Total images in {split}: {total_images_in_split}")



Split: train
  Class: Benign -> 973 images
  Class: Healthy -> 2315 images
  Class: Malignant -> 414 images
Total images in train: 3702

Split: test
  Class: Benign -> 132 images
  Class: Healthy -> 329 images
  Class: Malignant -> 62 images
Total images in test: 523

Split: valid
  Class: Benign -> 288 images
  Class: Healthy -> 658 images
  Class: Malignant -> 112 images
Total images in valid: 1058


In [10]:
from ultralytics import YOLO

model = YOLO("yolo11n-cls.pt")

results = model.train(
    data='/content/BreastCancerv2-1',
    epochs=20,
    imgsz=128,
    augment = True
)

Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/BreastCancerv2-1, epochs=20, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning /content/BreastCancerv2-1/train... 3702 images, 0 corrupt: 100%|██████████| 3702/3702 [00:00<00:00, 5173.48it/s]

train: New cache created: /content/BreastCancerv2-1/train.cache



val: Scanning /content/BreastCancerv2-1/test... 523 images, 0 corrupt: 100%|██████████| 523/523 [00:00<00:00, 2322.94it/s]

val: New cache created: /content/BreastCancerv2-1/test.cache


...

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.
WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc:  88%|████████▊ | 15/17 [00:01<00:00,  7.40it/s]

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.
WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc: 100%|██████████| 17/17 [00:01<00:00,  8.93it/s]


                   all      0.994          1
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


In [11]:
from ultralytics import YOLO


In [12]:
model = YOLO("/content/runs/classify/train2/weights/best.pt")


In [13]:
results = model.predict(source="/content/b1.jpg")



image 1/1 /content/b1.jpg: 128x128 Benign 1.00, Healthy 0.00, Malignant 0.00, 10.7ms
Speed: 4.0ms preprocess, 10.7ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)


In [14]:
for result in results:
    predicted_class_index = result.probs.top1
    predicted_class_name = result.names[predicted_class_index]
    confidence = result.probs.top1conf

    print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")

Predicted class: Benign, Confidence: 1.00


In [15]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from PIL import Image


test_folder = "/content/BreastCancerv2-1/test"

def load_test_data(test_folder):
    images = []
    true_labels = []
    class_names = os.listdir(test_folder)
    label_map = {class_name: idx for idx, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_folder = os.path.join(test_folder, class_name)
        if os.path.isdir(class_folder):
            for image_file in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_file)
                img = cv2.imread(image_path)
                img = cv2.resize(img, (416, 416))
                images.append(img)
                true_labels.append(label_map[class_name])

    return np.array(images), np.array(true_labels), label_map

def get_predictions(test_folder):
    images, true_labels, label_map = load_test_data(test_folder)
    predicted_labels = []

    for img, true_label in zip(images, true_labels):
        temp_image_path = "/tmp/temp_image.jpg"
        cv2.imwrite(temp_image_path, img)

        results = model.predict(temp_image_path, save=False, verbose=False)

        if results:
            predicted_class_index = results[0].probs.top1
            predicted_class_name = results[0].names[predicted_class_index]
            predicted_label = label_map.get(predicted_class_name, -1)
            predicted_labels.append(predicted_label)

    return true_labels, predicted_labels, list(label_map.keys())

def generate_confusion_matrix_image(true_labels, predicted_labels, class_names):
    cm = confusion_matrix(true_labels, predicted_labels)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names, cbar=False)

    plt.xlabel('Predicted Class')
    plt.ylabel('True Class')
    plt.title('Confusion Matrix')

    plt.savefig('/content/Confusion_Matrix.png')
    plt.close()

    print("Confusion Matrix Image saved as 'Confusion_Matrix.png'")

def get_accuracy(true_labels, predicted_labels):
    accuracy = (np.array(true_labels) == np.array(predicted_labels)).mean()
    return accuracy

true_labels, predicted_labels, class_names = get_predictions(test_folder)

generate_confusion_matrix_image(true_labels, predicted_labels, class_names)

accuracy = get_accuracy(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Confusion Matrix Image saved as 'Confusion_Matrix.png'
Accuracy: 99.43%
